# Dimensionality Recution
Typically, when we train we have instances that have trillions upon billions of features per instance. Not only does it make training really slow, it could also make it harder to find a good solution. This is often refered as the _curse of dimensionality_.

Fortunatly for us, it's often possible to reduce the dimensions!!! For example, the edges of the MNIST set could be trimmed since they hold no useful information. If two neighboring pixels are highly correlated, they can be merged into a single pixel. Keep in mind that reducing features leads to information loss! This may lead to slightly worse performance. That aside, dimensionality reduction can lead to data visualization(AKA _DataViz_). Here we will look at three techniques: PCA, Kernel PCA, and LLE.

## The Curse of Dimensionality
High dimensional stuff be cray cray fam. Look at the book for diz. Keep in mind, the greater the dimensions, the greater the risk of overfitting it! One theoretical solution would be to increase the training set to a rediculous size... however this in practice is not possible.

## Main Approaches for Dimensionality Reduction
Before we look at specific reduction algos, let's look at two main approaches: projection and Manifold Learning

### Projection
In most real-world problems, training instances are _not_ spread out uniformly across all dimensions. Many features are almost constant, while others are highly correlated, hence they actually lie within a lower-dimensional _subspace_. Projection is not the best approach however. Take a look at the famous _Swiss roll_ toy dataset. Dropping a plane is bad, you need to unroll it!

### Manifold Learning
Many dimensionality reduction algos work by modeling the _manifold_ on which the training instances lie upon. This is called _Manifold Learning_. This relies onthe _manifold assumtion_, also called _manifold hypothesis_, which holds that most real-world highdimensional data lie closer to a much lower-dimensional manifold.

Again, think about the MNIST set: all handwriten digets have some similarities. They have connected lines, white borders, and are more or less centered. If we had instead randomly generated images, we probably wouldn't be able to reduce the dimensions, given that constraints tend to lead to a dataset being projected onto a lower dimension manifold.

The manifold assumption also has another assumption being that the task at hand will be simpler if expressed in a lower dimension. In short, it all depends on how the data set is!

## PCA
_Principal Component Analysis_(PCA) is the most popular dimensionality reduction algo. It identifies the hyperplane that lies closest to the data, then projects the data onto it.

### Preserving the Variance
You need to choose the right plane and axis to maintain the variance! This is to minizmize the mean squared distance between the original and it's projection onto the axis. This is the idea behind PCA.

### Principal Components
